<a href="https://colab.research.google.com/github/Muku94/Covid_19/blob/main/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# importing dependencies

import os
import stat
import pathlib
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from google.colab import files
from sklearn.metrics import classification_report,confusion_matrix

from tensorflow.keras.applications.vgg16 import preprocess_input

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mukuwa","key":"1a0b348871f02a5f3bf98f32ce62ba2b"}'}

In [4]:
# Path to your kaggle.json file
kaggle_json_path = r"/content/kaggle.json"

# Ensure the Kaggle API key file has the correct permissions
os.chmod(kaggle_json_path, stat.S_IRUSR | stat.S_IWUSR)

In [5]:
# setting the environment

os.environ["KAGGLE_CONFIG_DIR"] = "/content"

In [6]:
!kaggle datasets download -d sid321axn/covid-cxr-image-dataset-research

Dataset URL: https://www.kaggle.com/datasets/sid321axn/covid-cxr-image-dataset-research
License(s): CC0-1.0
100% 571M/572M [00:29<00:00, 25.8MB/s]
100% 572M/572M [00:29<00:00, 20.2MB/s]


In [7]:
## unzipping the downloaded files

file_name = '/content/covid-cxr-image-dataset-research.zip'
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall()

In [13]:
data_path = '/content/COVID_IEEE'

for dirpath,dirnames,filenames in os.walk(data_path):
    print(f"there are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

there are 3 directories and 0 images in '/content/COVID_IEEE'.
there are 0 directories and 619 images in '/content/COVID_IEEE/virus'.
there are 0 directories and 536 images in '/content/COVID_IEEE/covid'.
there are 0 directories and 668 images in '/content/COVID_IEEE/normal'.


In [14]:
data_dir = pathlib.Path(data_path)
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names

array(['covid', 'normal', 'virus'], dtype='<U6')

In [15]:
# Function to load images and labels from a directory
def load_images_from_folder(folder_path, label, sample_size):
    images = []
    labels = []
    files = os.listdir(folder_path)

    # Randomly sample files
    files = random.sample(files, sample_size)

    for file in files:
        img_path = os.path.join(folder_path, file)
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, (224, 224))
            images.append(image)
            labels.append(label)
    return images, labels

# Define folder paths for each class
covid_folder = "/content/COVID_IEEE/covid"
normal_folder = "/content/COVID_IEEE/normal"
pneumonia_folder = "/content/COVID_IEEE/virus"

# Count the number of files in each class directory
num_covid = len(os.listdir(covid_folder))
num_normal = len(os.listdir(normal_folder))
num_pneumonia = len(os.listdir(pneumonia_folder))

# Determine the minimum count
min_count = min(num_covid, num_normal, num_pneumonia)

# Load images and labels for each class
covid_images, covid_labels = load_images_from_folder(covid_folder, 0, min_count)
normal_images, normal_labels = load_images_from_folder(normal_folder, 1, min_count)
pneumonia_images, pneumonia_labels = load_images_from_folder(pneumonia_folder, 2, min_count)

# Combine all images and labels
data = covid_images + normal_images + pneumonia_images
labels = covid_labels + normal_labels + pneumonia_labels

# Shuffle the dataset (optional)
#combined = list(zip(data, labels))
#random.shuffle(combined)
#data[:], labels[:] = zip(*combined)

# Convert lists to numpy arrays if required for your model
data = np.array(data)
labels = np.array(labels)

print(f"Number of images in each class: {min_count}")
print(f"Total number of images: {len(data)}")

Number of images in each class: 536
Total number of images: 1608


In [17]:
# saving the np.files

np.save('data.npy', data)
np.save('labels.npy', labels)

# adding a txt files that describes the data set & labbles

with open('data_info.txt', 'w') as f:
    f.write(f"Number of images in each class: {min_count}\n")
    f.write(f"Total number of images: {len(data)}\n")



In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [21]:
# Function to load images and labels from a directory
def load_images_from_folder(folder_path, label, sample_size):
    images = []
    labels = []
    files = os.listdir(folder_path)

    # Randomly sample files
    files = random.sample(files, sample_size)

    for file in files:
        img_path = os.path.join(folder_path, file)
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, (224, 224))
            images.append(image)
            labels.append(label)
    return images, labels

# Define folder paths for each class
covid_folder = "/content/COVID_IEEE/covid/"
normal_folder = "/content/COVID_IEEE/normal/"
pneumonia_folder = "/content/COVID_IEEE/virus"

# Count the number of files in each class directory
num_covid = len(os.listdir(covid_folder))
num_normal = len(os.listdir(normal_folder))
num_pneumonia = len(os.listdir(pneumonia_folder))

# Determine the minimum count
min_count = min(num_covid, num_normal, num_pneumonia)

# Load images and labels for each class
covid_images, covid_labels = load_images_from_folder(covid_folder, 0, min_count)
normal_images, normal_labels = load_images_from_folder(normal_folder, 1, min_count)
pneumonia_images, pneumonia_labels = load_images_from_folder(pneumonia_folder, 2, min_count)

# Combine all images and labels
data = covid_images + normal_images + pneumonia_images
labels = covid_labels + normal_labels + pneumonia_labels

# Shuffle the dataset (optional)
combined = list(zip(data, labels))
random.shuffle(combined)
data[:], labels[:] = zip(*combined)

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Preprocess the data using VGG16's preprocess_input
data = preprocess_input(data)

# Save the data and labels arrays to .npy files
np.save('/content/data.npy', data)
np.save('/content/labels.npy', labels)

print(f"Number of images in each class: {min_count}")
print(f"Total number of images: {len(data)}")

Number of images in each class: 536
Total number of images: 1608


In [22]:
data

array([[[[-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         [-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         [-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         ...,
         [-6.7939003e+01, -8.0778999e+01, -8.7680000e+01],
         [-7.4939003e+01, -8.7778999e+01, -9.4680000e+01],
         [-7.2939003e+01, -8.5778999e+01, -9.2680000e+01]],

        [[-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         [-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         [-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         ...,
         [-5.7939003e+01, -7.0778999e+01, -7.7680000e+01],
         [-6.8939003e+01, -8.1778999e+01, -8.8680000e+01],
         [-6.7939003e+01, -8.0778999e+01, -8.7680000e+01]],

        [[-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         [-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         [-1.0393900e+02, -1.1677900e+02, -1.2368000e+02],
         ...,
         [-6.0939003e+01, -7.3778999e+01, -8.0680000e+01],
         [